<a href="https://colab.research.google.com/github/pedro00dk/ufpe-security-analysis/blob/master/ufpe_sa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UFPE Security Analysis

Fetching github repo to get database file.

In [1]:
!rm --recursive --force -- ./ufpe-security-analysis/
!git clone -- https://github.com/pedro00dk/ufpe-security-analysis
!ls -- ./ufpe-security-analysis/

Cloning into 'ufpe-security-analysis'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 24 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.
 README.md   ufpe_sa.ipynb  'UFPE SSI.xlsx'


Install necessary tools for loading excel

In [2]:
!pip install xlrd

In [0]:
import numpy as np
import pandas as pd
import re

The database consists in an excel file with sheets per year of information. All of these sheets have to be loaded. I printed only a single line of the dataframs to check the data formats and column names for concatenation.

In [4]:
xls = pd.ExcelFile('./ufpe-security-analysis/UFPE SSI.xlsx')
dfs = xls.parse(sheet_name=xls.sheet_names)

[display(y, df.sample(1)) for y, df in dfs.items()]

'2008'

,Data,Quadrante,Local,Horário,Tipo,Descrição Sumária,Partes,Medidas
116,07.05.08,ZLI,Av. do Arquiteto,13h40,Colisão,Ocorrência 626- o carro FIAT KJR 5588 colidiu ...,Francisco Assis Gomes/Gozalo Henrique,Ñinf


'2009'

,Data,Quadrante,Local,Horário,Tipo,Descrição Sumária,Partes,Medidas
24,26.01.09,Gama,Castelo d'água,18h05,Registro de ocorrência,Ocorrência 1132 - 01 indivíduo pichando no local.,Estudante de Filosofia Raphael Soares Lucas do...,O GTO foi ao local.


'2011'

...

'2014'

,Data,Quadrante,Local,horário,Tipo,Descrição Sumária,Partes,Medidas
281,2014-08-04 00:00:00,Gama,BC,Ñ inf,Registro de ocorrência,Ocorrência 7705 - segundo registro funcionária...,Cláudia da Silva.,Ñinf


'2015'

,Data,Quadrante,Local,horário,Tipo,Descrição Sumária,Partes,Medidas,Unnamed: 8,Unnamed: 9,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
170,2015-04-09,Beta,Área II,21h40,Prestação de Socorro,"Ocorrência 0198 TURMA ""B"" - segundo registro u...",Ñ inf,A aluna foi conduzida até a UPA da Caxangá.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


[None, None, None, None, None, None, None]

In [5]:
# lower all columns and remove extra spaces
dfs = {y: df.rename(lambda c: c.lower().strip(), axis='columns') for y, df in dfs.items()}

# drop columns
drop_rule = re.compile(r'(?x) descrição \s sumária | medidas | unnamed.*')
dfs = {y: df.drop([c for c in df.columns if drop_rule.match(c)], axis='columns') for y, df in dfs.items()}

# unify null elements
na_rule = re.compile(r'(?i)ñ.*')
dfs = {y: df.replace(na_rule.pattern, np.nan, regex=True) for y, df in dfs.items()}

# unify date columns, date have different formats along all dataframes, so they have to be processed before concat
date_format = {
    '2008': '%d.%m.%y',
    '2009': '%d.%m.%y',
    '2011': '%d.%m.%y',
    '2012': '%d.%m.%y',
    '2013': '%Y-%m-%d',
    '2014': '%Y-%m-%d', #'%Y-%m-%d %h:%m:$s'
    '2015': '%Y-%m-%d'
}
dfs = {
    y: df.assign(**{'data': lambda df: pd.to_datetime(df['data'], errors='coerce', format=date_format[y], exact=False)})
    for y, df in dfs.items()
}

# merge dataframes
df = pd.concat(dfs, ignore_index=True)
df.sample(40)

,data,quadrante,local,horário,tipo,partes
573,2011-12-07,Delta,CCS,NaN,Carro c/ Vulnerabilidade,NaN
720,2012-07-21,Gama,Est.do CCSA próximo a Hotelaria,NaN,Registro de ocorrência,NaN
1936,2015-05-20,Gama,Estacionamento do CCSA,12h50,Registro de Ocorrência,Professora: Maria Conceição Torres.
1756,2015-02-09,Beta,Em frente ao CIN,NaN,Registro de Ocorrência,NaN
258,2009-02-18,NaN,NaN,NaN,Registro de ocorrência,NaN
708,2012-06-28,Alfa,NAE.,NaN,Dano Patrimonio,NaN
1918,2015-05-10,Delta,CIS,18h25,Registro de Ocorrência,NaN
967,2013-04-03,Beta,CCEN,NaN,Furto privado,Leandro Chves Rego (vítima)
904,2013-02-16,NaN,NaN,14h45,Prestação de socorro,Edna Maria dos Santos
747,2012-09-04,Alfa,CEU - Masc,01h30,Registro de ocorrência,NaN


In [7]:
df['quadrante'] = df['quadrante']\
    .fillna('')\
    .apply(lambda e: str(e).strip().lower())\
    .apply(lambda e: 'None' if len(e) <= 1 else e)\
    .apply(lambda e: 'externa' if 'externa' in e else e)\
    .apply(lambda e: 'externa' if e.startswith('z') and e.endswith('e') else e)\
    .apply(lambda e: 'interna' if e.startswith('z') else e)\
    .apply(lambda e: e[:e.index('/')] if '/' in e else e)

places_rules = [
    ('CAC-CFCH', re.compile(r'(?ix) cfch | cac | (a|á)rtes | gere | sintuf')),
    ('CE',       re.compile(r'(?ix) ((^|\s) ce ($|\s)) | col(é|e)gio | aplica | educaç | ((^|\s)cap($|\s))')),
    ('CCSA',     re.compile(r'(?ix) ccsa | cssa | adu(e?)fpe | economia | lamsa | ((^|\s)de(q|c)($|\s)) | hotel | lac | turism | qu(í|i)mic')),
    ('CTG',      re.compile(r'(?ix) ctg | oce(o|a)no | museu | fae | eng. | engenh')),
    ('CCEN',     re.compile(r'(?ix) ccen | dep .* físi | matem')),
    ('CCB',      re.compile(r'(?ix) ccb')),
    ('CCS',      re.compile(r'(?ix) ccs | odonto | dermato | medic | fono | histono | farmac | nutri | anato | faci | biot | buco | psico | svo | sa(ú|u)de | antib | nasc')),
    ('CIN',      re.compile(r'(?ix) cin | inform')),
    ('AREA2',    re.compile(r'(?ix) ((á|a)rea \s? (2|ii)) | pcu | prefeit | den')),
    ('RU',       re.compile(r'(?ix) ru | restaurante')),
    ('NTI',      re.compile(r'(?ix) ((^|\s) nti) | n(ú|u)cleo de tecn')),
    ('NEDF',     re.compile(r'(?ix) nedf | nefd | dine | nai | fisio | terapi | (educ .* ica) | arqueo | n(ú|u)cleo | quadra')),
    ('LANSA',    re.compile(r'(?ix) lansa')),
    ('CECINE',   re.compile(r'(?ix) cecine')),
    ('FARMACIA', re.compile(r'(?ix) f(á|a)rm(á|a)cia')),
    ('CEU',      re.compile(r'(?ix) ((^|\s) ceu ($|\s|-)) | masc | femin')),
    ('DGOS',     re.compile(r'(?ix) dgos')),
    ('REITORIA', re.compile(r'(?ix) reitoria | proa | caixa | brasi | ((^|\s)nae($|\s|\.)) | proacad')),
    ('BR101',    re.compile(r'(?ix) (br .* 101?) | sudene | viadut | girad')),
    ('HC',       re.compile(r'(?ix) hc | hospital | cl(í|i)nicas')),
    ('ENTRADA',  re.compile(r'(?ix) entrada | sa(í|i)da | editora | (o|ô)nibus | guarita | ((^|\s)cos($|\s))')),
    ('AVENIDA',  re.compile(r'(?ix) avenida | av\. | ceco(n|m) | conven | central | arqui | reitore | teatr | ((^|\s)bc($|\s))')),
    ('CONCHA',   re.compile(r'(?ix) concha')),
    ('COOPER',   re.compile(r'(?ix) cooper')),
    ('CAST_AGUA',re.compile(r'(?ix) castelo | agua')),
    ('CASA-ESTD',re.compile(r'(?ix) casa | mist')),
    ('CLUBE',    re.compile(r'(?ix) clube')),
    ('LAKE',     re.compile(r'(?ix) lago | lagu | riach | cavouc')),
    ('CAMPUS',   re.compile(r'(?ix) campus | ufpe')),
    ('TVU',      re.compile(r'(?ix) tvu | ((^|\s)tv($|\s)) | tv \s u | r(a|á)dio')),
    ('none',     re.compile(r'(?ix) none')),
    ('OUTROS',   re.compile(r'(?ix) .*')),
]
place_matcher = lambda e: [place for place, rule in places_rules if rule.search(e)][0]
df['local'] = df['local']\
    .fillna('None')\
    .apply(lambda e: str(e).strip().lower())\
    .apply(place_matcher)
df.sample(10)

,data,quadrante,local,horário,tipo,partes
1328,2014-01-24,None,none,NaN,Registro de ocorrência,Alcione Honório(segurança condutor da HALLEUX ...
347,2009-10-05,None,CIN,17h40,Furto,GTO
1280,2013-11-28,delta,CCB,09h50,Registro de ocorrência,Henrique Augusto Alves da Costa
760,2012-09-19,delta,CCS,19h40,Roubo(ZLE),Gleyce Kelly
79,2008-03-11,gama,CAC-CFCH,NaN,Colisão,Padro Diego Rodrigues/Carlos Alberto Carneiro
455,2011-05-24,alfa,AREA2,17h30,Roubo (ZLE),NaN
1098,2013-06-19,interna,CAMPUS,NaN,Registro de ocorrência,NaN
147,2008-07-12,alfa,CEU,16h45,Registro de ocorrência,NaN
873,2013-01-23,None,OUTROS,23h00,Registro de ocorrência,....Antonio Souza.
1133,2013-07-05,alfa,CEU,19h50,Furto privado,Funcionário do CIN Eduardo Lima Tavares
